# Part B: Data Warehouse Modeling (Star Schema)

This notebook builds a Star Schema data warehouse in SQLite from the cleaned retail sales data.

**Schema Design:**
- **FactSales** (fact table): sale_id, date_id, product_id, customer_id, quantity, price, total_spent, payment_method, location
- **DimDate** (dimension): date_id, full_date, day_of_week, day_name, is_weekend, month, month_name, quarter, year, fiscal_year
- **DimProduct** (dimension): product_id, product_name, category
- **DimCustomer** (dimension): customer_id, customer_name, total_transactions, total_spent, avg_basket_size, preferred_category, preferred_location

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

DATA_DIR = os.path.join('..', 'data')
WAREHOUSE_DIR = os.path.join('..', 'warehouse')
CLEAN_FILE = os.path.join(DATA_DIR, 'cleaned_sales.csv')
DB_FILE = os.path.join(WAREHOUSE_DIR, 'sales_warehouse.db')

# Remove old database if it exists (for re-runnability)
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)

In [2]:
# Load cleaned data
df = pd.read_csv(CLEAN_FILE, parse_dates=['Transaction Date'])
print(f'Loaded {len(df)} rows')
df.head()

Loaded 11362 rows


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_Pat,18.5,10,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_Milk,29.0,9,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_But,21.5,2,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_Bev,27.5,9,247.5,Credit Card,Online,2022-05-07,True
4,TXN_4575373,CUST_05,Food,Item_6_Food,12.5,7,87.5,Digital Wallet,Online,2022-10-02,False


## 1. Build Dimension Tables

### DimDate

Pre-calculates date attributes: `is_weekend`, `month_name`, `quarter`, `fiscal_year`.
Fiscal year convention: starts in January (fiscal_year = calendar year).

In [3]:
# Extract unique dates and build the Date dimension
unique_dates = df['Transaction Date'].dt.date.unique()
unique_dates = sorted(unique_dates)

dim_date = pd.DataFrame({
    'date_id': range(1, len(unique_dates) + 1),
    'full_date': unique_dates
})

dim_date['full_date'] = pd.to_datetime(dim_date['full_date'])
dim_date['day_of_week'] = dim_date['full_date'].dt.dayofweek  # Monday=0, Sunday=6
dim_date['day_name'] = dim_date['full_date'].dt.day_name()
dim_date['is_weekend'] = (dim_date['day_of_week'] >= 5).astype(int)  # Saturday=5, Sunday=6
dim_date['month'] = dim_date['full_date'].dt.month
dim_date['month_name'] = dim_date['full_date'].dt.month_name()
dim_date['quarter'] = dim_date['full_date'].dt.quarter
dim_date['year'] = dim_date['full_date'].dt.year
dim_date['fiscal_year'] = dim_date['full_date'].dt.year  # Fiscal year = calendar year

print(f'DimDate: {len(dim_date)} rows')
dim_date.head(10)

DimDate: 1114 rows


,date_id,full_date,day_of_week,day_name,is_weekend,month,month_name,quarter,year,fiscal_year
0,1,2022-01-01,5,Saturday,1,1,January,1,2022,2022
1,2,2022-01-02,6,Sunday,1,1,January,1,2022,2022
2,3,2022-01-03,0,Monday,0,1,January,1,2022,2022
3,4,2022-01-04,1,Tuesday,0,1,January,1,2022,2022
4,5,2022-01-05,2,Wednesday,0,1,January,1,2022,2022
5,6,2022-01-06,3,Thursday,0,1,January,1,2022,2022
6,7,2022-01-07,4,Friday,0,1,January,1,2022,2022
7,8,2022-01-08,5,Saturday,1,1,January,1,2022,2022
8,9,2022-01-09,6,Sunday,1,1,January,1,2022,2022
9,10,2022-01-10,0,Monday,0,1,January,1,2022,2022


### DimProduct

Extracts unique products with their category.

In [4]:
# Extract unique (Item, Category) pairs and assign surrogate keys
products = df[['Item', 'Category']].drop_duplicates().sort_values('Item').reset_index(drop=True)

dim_product = pd.DataFrame({
    'product_id': range(1, len(products) + 1),
    'product_name': products['Item'].values,
    'category': products['Category'].values
})

print(f'DimProduct: {len(dim_product)} rows')
dim_product.head(10)

DimProduct: 200 rows


,product_id,product_name,category
0,1,Item_10_Bev,Beverages
1,2,Item_10_But,Butchers
2,3,Item_10_Cea,Computers And Electric Accessories
3,4,Item_10_Ehe,Electric Household Essentials
4,5,Item_10_Food,Food
5,6,Item_10_Fur,Furniture
6,7,Item_10_Milk,Milk Products
7,8,Item_10_Pat,Patisserie
8,9,Item_11_Bev,Beverages
9,10,Item_11_But,Butchers


### DimCustomer

The raw dataset does not include demographic attributes (gender, age, etc.) for customers --
only a `Customer ID` identifier (e.g., `CUST_01`). To make this dimension table meaningful,
we derive behavioral attributes from the sales data: total transactions, total spending,
average basket size, preferred product category, and preferred shopping location.

In [5]:
# Derive behavioral attributes for each customer from the sales data
customer_stats = df.groupby('Customer ID').agg(
    total_transactions=('Transaction ID', 'nunique'),
    total_spent=('Total Spent', 'sum'),
    avg_basket_size=('Quantity', 'mean'),
).reset_index()

# Preferred category: the category with the most purchases per customer
preferred_cat = (
    df.groupby(['Customer ID', 'Category']).size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .drop_duplicates(subset='Customer ID', keep='first')
    .rename(columns={'Category': 'preferred_category'})[['Customer ID', 'preferred_category']]
)

# Preferred location: the location with the most purchases per customer
preferred_loc = (
    df.groupby(['Customer ID', 'Location']).size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .drop_duplicates(subset='Customer ID', keep='first')
    .rename(columns={'Location': 'preferred_location'})[['Customer ID', 'preferred_location']]
)

customer_stats = customer_stats.merge(preferred_cat, on='Customer ID').merge(preferred_loc, on='Customer ID')
customer_stats = customer_stats.sort_values('Customer ID').reset_index(drop=True)

dim_customer = pd.DataFrame({
    'customer_id': range(1, len(customer_stats) + 1),
    'customer_name': customer_stats['Customer ID'].values,
    'total_transactions': customer_stats['total_transactions'].values,
    'total_spent': customer_stats['total_spent'].values.round(2),
    'avg_basket_size': customer_stats['avg_basket_size'].values.round(2),
    'preferred_category': customer_stats['preferred_category'].values,
    'preferred_location': customer_stats['preferred_location'].values,
})

print(f'DimCustomer: {len(dim_customer)} rows')
dim_customer

DimCustomer: 25 rows


,customer_id,customer_name,total_transactions,total_spent,avg_basket_size,preferred_category,preferred_location
0,1,CUST_01,461,55908.0,5.51,Computers And Electric Accessories,Online
1,2,CUST_02,447,59512.5,5.61,Electric Household Essentials,In-Store
2,3,CUST_03,423,57150.0,5.65,Butchers,Online
3,4,CUST_04,428,57959.0,5.60,Butchers,In-Store
4,5,CUST_05,497,63855.5,5.40,Beverages,Online
5,6,CUST_06,443,56020.0,5.46,Computers And Electric Accessories,In-Store
6,7,CUST_07,433,57084.5,5.53,Butchers,Online
7,8,CUST_08,480,62850.5,5.70,Beverages,Online
8,9,CUST_09,465,57113.0,5.36,Furniture,Online
9,10,CUST_10,458,60367.5,5.54,Computers And Electric Accessories,In-Store


## 2. Build Fact Table (FactSales)

Maps each sale row to its dimension keys via lookups.

In [6]:
# Create lookup dictionaries for dimension keys
date_lookup = dict(zip(dim_date['full_date'].dt.date, dim_date['date_id']))
product_lookup = dict(zip(dim_product['product_name'], dim_product['product_id']))
customer_lookup = dict(zip(dim_customer['customer_name'], dim_customer['customer_id']))

# Build the fact table
fact_sales = pd.DataFrame({
    'sale_id': range(1, len(df) + 1),
    'date_id': df['Transaction Date'].dt.date.map(date_lookup).values,
    'product_id': df['Item'].map(product_lookup).values,
    'customer_id': df['Customer ID'].map(customer_lookup).values,
    'quantity': df['Quantity'].values,
    'price': df['Price Per Unit'].values,
    'total_spent': df['Total Spent'].values,
    'payment_method': df['Payment Method'].values,
    'location': df['Location'].values
})

# Verify no null foreign keys
print(f'FactSales: {len(fact_sales)} rows')
print(f'Null FK counts: date_id={fact_sales["date_id"].isna().sum()}, '
      f'product_id={fact_sales["product_id"].isna().sum()}, '
      f'customer_id={fact_sales["customer_id"].isna().sum()}')
fact_sales.head(10)

FactSales: 11362 rows
Null FK counts: date_id=0, product_id=0, customer_id=0


,sale_id,date_id,product_id,customer_id,quantity,price,total_spent,payment_method,location
0,1,829,8,9,10,18.5,185.0,Digital Wallet,Online
1,2,569,63,22,9,29.0,261.0,Digital Wallet,Online
2,3,278,18,2,2,21.5,43.0,Credit Card,Online
3,4,127,49,6,9,27.5,247.5,Credit Card,Online
4,5,275,173,5,7,12.5,87.5,Digital Wallet,Online
5,6,526,85,7,8,5.0,40.0,Credit Card,In-Store
6,7,481,54,23,1,27.5,27.5,Credit Card,In-Store
7,8,804,106,25,3,36.5,109.5,Cash,Online
8,9,1079,146,22,9,8.0,72.0,Cash,In-Store
9,10,617,141,23,7,6.5,45.5,Cash,Online


## 3. Create SQLite Database

Create tables with PRIMARY KEY and FOREIGN KEY constraints, then insert the data.

In [7]:
conn = sqlite3.connect(DB_FILE)
conn.execute('PRAGMA foreign_keys = ON;')
cursor = conn.cursor()

# Create DimDate table
cursor.execute('''
    CREATE TABLE DimDate (
        date_id     INTEGER PRIMARY KEY,
        full_date   DATE NOT NULL,
        day_of_week INTEGER NOT NULL,
        day_name    TEXT NOT NULL,
        is_weekend  INTEGER NOT NULL,
        month       INTEGER NOT NULL,
        month_name  TEXT NOT NULL,
        quarter     INTEGER NOT NULL,
        year        INTEGER NOT NULL,
        fiscal_year INTEGER NOT NULL
    );
''')

# Create DimProduct table
cursor.execute('''
    CREATE TABLE DimProduct (
        product_id   INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category     TEXT NOT NULL
    );
''')

# Create DimCustomer table (with derived behavioral attributes)
cursor.execute('''
    CREATE TABLE DimCustomer (
        customer_id        INTEGER PRIMARY KEY,
        customer_name      TEXT NOT NULL,
        total_transactions INTEGER NOT NULL,
        total_spent        REAL NOT NULL,
        avg_basket_size    REAL NOT NULL,
        preferred_category TEXT NOT NULL,
        preferred_location TEXT NOT NULL
    );
''')

# Create FactSales table with foreign keys
cursor.execute('''
    CREATE TABLE FactSales (
        sale_id        INTEGER PRIMARY KEY,
        date_id        INTEGER NOT NULL,
        product_id     INTEGER NOT NULL,
        customer_id    INTEGER NOT NULL,
        quantity       INTEGER NOT NULL,
        price          REAL NOT NULL,
        total_spent    REAL NOT NULL,
        payment_method TEXT NOT NULL,
        location       TEXT NOT NULL,
        FOREIGN KEY (date_id) REFERENCES DimDate(date_id),
        FOREIGN KEY (product_id) REFERENCES DimProduct(product_id),
        FOREIGN KEY (customer_id) REFERENCES DimCustomer(customer_id)
    );
''')

conn.commit()
print('Tables created successfully.')

Tables created successfully.


In [8]:
# Insert data into dimension tables
dim_date_db = dim_date.copy()
dim_date_db['full_date'] = dim_date_db['full_date'].dt.strftime('%Y-%m-%d')
dim_date_db.to_sql('DimDate', conn, if_exists='append', index=False)

dim_product.to_sql('DimProduct', conn, if_exists='append', index=False)
dim_customer.to_sql('DimCustomer', conn, if_exists='append', index=False)

# Insert data into fact table
fact_sales.to_sql('FactSales', conn, if_exists='append', index=False)

conn.commit()
print('Data inserted successfully.')

Data inserted successfully.


## 4. Verify Data Integrity

In [9]:
# Verify row counts
tables = ['DimDate', 'DimProduct', 'DimCustomer', 'FactSales']
for table in tables:
    count = pd.read_sql(f'SELECT COUNT(*) as cnt FROM {table}', conn)['cnt'][0]
    print(f'{table}: {count} rows')

DimDate: 1114 rows
DimProduct: 200 rows
DimCustomer: 25 rows
FactSales: 11362 rows


In [10]:
# Verify referential integrity: all FKs in FactSales exist in dimension tables
orphan_dates = pd.read_sql('''
    SELECT COUNT(*) as cnt FROM FactSales f
    LEFT JOIN DimDate d ON f.date_id = d.date_id
    WHERE d.date_id IS NULL
''', conn)['cnt'][0]

orphan_products = pd.read_sql('''
    SELECT COUNT(*) as cnt FROM FactSales f
    LEFT JOIN DimProduct p ON f.product_id = p.product_id
    WHERE p.product_id IS NULL
''', conn)['cnt'][0]

orphan_customers = pd.read_sql('''
    SELECT COUNT(*) as cnt FROM FactSales f
    LEFT JOIN DimCustomer c ON f.customer_id = c.customer_id
    WHERE c.customer_id IS NULL
''', conn)['cnt'][0]

print(f'Orphan date_ids: {orphan_dates}')
print(f'Orphan product_ids: {orphan_products}')
print(f'Orphan customer_ids: {orphan_customers}')

if orphan_dates == 0 and orphan_products == 0 and orphan_customers == 0:
    print('\nAll foreign key references are valid!')

Orphan date_ids: 0
Orphan product_ids: 0
Orphan customer_ids: 0

All foreign key references are valid!


In [11]:
# Sample query: Join all tables and display sample rows
sample_query = '''
    SELECT 
        f.sale_id,
        d.full_date,
        d.day_name,
        d.is_weekend,
        d.month_name,
        d.quarter,
        d.fiscal_year,
        p.product_name,
        p.category,
        c.customer_name,
        f.quantity,
        f.price,
        f.total_spent,
        f.payment_method,
        f.location
    FROM FactSales f
    JOIN DimDate d ON f.date_id = d.date_id
    JOIN DimProduct p ON f.product_id = p.product_id
    JOIN DimCustomer c ON f.customer_id = c.customer_id
    LIMIT 15;
'''
sample = pd.read_sql(sample_query, conn)
print('Sample of joined warehouse data:')
sample

Sample of joined warehouse data:


,sale_id,full_date,day_name,is_weekend,month_name,quarter,fiscal_year,product_name,category,customer_name,quantity,price,total_spent,payment_method,location
0,1,2024-04-08,Monday,0,April,2,2024,Item_10_Pat,Patisserie,CUST_09,10,18.5,185.0,Digital Wallet,Online
1,2,2023-07-23,Sunday,1,July,3,2023,Item_17_Milk,Milk Products,CUST_22,9,29.0,261.0,Digital Wallet,Online
2,3,2022-10-05,Wednesday,0,October,4,2022,Item_12_But,Butchers,CUST_02,2,21.5,43.0,Credit Card,Online
3,4,2022-05-07,Saturday,1,May,2,2022,Item_16_Bev,Beverages,CUST_06,9,27.5,247.5,Credit Card,Online
4,5,2022-10-02,Sunday,1,October,4,2022,Item_6_Food,Food,CUST_05,7,12.5,87.5,Digital Wallet,Online
5,6,2023-06-10,Saturday,1,June,2,2023,Item_1_Food,Food,CUST_07,8,5.0,40.0,Credit Card,In-Store
6,7,2023-04-26,Wednesday,0,April,2,2023,Item_16_Fur,Furniture,CUST_23,1,27.5,27.5,Credit Card,In-Store
7,8,2024-03-14,Thursday,0,March,1,2024,Item_22_But,Butchers,CUST_25,3,36.5,109.5,Cash,Online
8,9,2024-12-14,Saturday,1,December,4,2024,Item_3_But,Butchers,CUST_22,9,8.0,72.0,Cash,In-Store
9,10,2023-09-09,Saturday,1,September,3,2023,Item_2_Food,Food,CUST_23,7,6.5,45.5,Cash,Online


In [12]:
# Display each table's sample for screenshots
print('=== DimDate (first 10 rows) ===')
display(pd.read_sql('SELECT * FROM DimDate LIMIT 10', conn))

print('\n=== DimProduct (first 10 rows) ===')
display(pd.read_sql('SELECT * FROM DimProduct LIMIT 10', conn))

print('\n=== DimCustomer (all rows) ===')
display(pd.read_sql('SELECT * FROM DimCustomer', conn))

print('\n=== FactSales (first 10 rows) ===')
display(pd.read_sql('SELECT * FROM FactSales LIMIT 10', conn))

=== DimDate (first 10 rows) ===


,date_id,full_date,day_of_week,day_name,is_weekend,month,month_name,quarter,year,fiscal_year
0,1,2022-01-01,5,Saturday,1,1,January,1,2022,2022
1,2,2022-01-02,6,Sunday,1,1,January,1,2022,2022
2,3,2022-01-03,0,Monday,0,1,January,1,2022,2022
3,4,2022-01-04,1,Tuesday,0,1,January,1,2022,2022
4,5,2022-01-05,2,Wednesday,0,1,January,1,2022,2022
5,6,2022-01-06,3,Thursday,0,1,January,1,2022,2022
6,7,2022-01-07,4,Friday,0,1,January,1,2022,2022
7,8,2022-01-08,5,Saturday,1,1,January,1,2022,2022
8,9,2022-01-09,6,Sunday,1,1,January,1,2022,2022
9,10,2022-01-10,0,Monday,0,1,January,1,2022,2022



=== DimProduct (first 10 rows) ===


,product_id,product_name,category
0,1,Item_10_Bev,Beverages
1,2,Item_10_But,Butchers
2,3,Item_10_Cea,Computers And Electric Accessories
3,4,Item_10_Ehe,Electric Household Essentials
4,5,Item_10_Food,Food
5,6,Item_10_Fur,Furniture
6,7,Item_10_Milk,Milk Products
7,8,Item_10_Pat,Patisserie
8,9,Item_11_Bev,Beverages
9,10,Item_11_But,Butchers



=== DimCustomer (all rows) ===


,customer_id,customer_name,total_transactions,total_spent,avg_basket_size,preferred_category,preferred_location
0,1,CUST_01,461,55908.0,5.51,Computers And Electric Accessories,Online
1,2,CUST_02,447,59512.5,5.61,Electric Household Essentials,In-Store
2,3,CUST_03,423,57150.0,5.65,Butchers,Online
3,4,CUST_04,428,57959.0,5.60,Butchers,In-Store
4,5,CUST_05,497,63855.5,5.40,Beverages,Online
5,6,CUST_06,443,56020.0,5.46,Computers And Electric Accessories,In-Store
6,7,CUST_07,433,57084.5,5.53,Butchers,Online
7,8,CUST_08,480,62850.5,5.70,Beverages,Online
8,9,CUST_09,465,57113.0,5.36,Furniture,Online
9,10,CUST_10,458,60367.5,5.54,Computers And Electric Accessories,In-Store



=== FactSales (first 10 rows) ===


,sale_id,date_id,product_id,customer_id,quantity,price,total_spent,payment_method,location
0,1,829,8,9,10,18.5,185.0,Digital Wallet,Online
1,2,569,63,22,9,29.0,261.0,Digital Wallet,Online
2,3,278,18,2,2,21.5,43.0,Credit Card,Online
3,4,127,49,6,9,27.5,247.5,Credit Card,Online
4,5,275,173,5,7,12.5,87.5,Digital Wallet,Online
5,6,526,85,7,8,5.0,40.0,Credit Card,In-Store
6,7,481,54,23,1,27.5,27.5,Credit Card,In-Store
7,8,804,106,25,3,36.5,109.5,Cash,Online
8,9,1079,146,22,9,8.0,72.0,Cash,In-Store
9,10,617,141,23,7,6.5,45.5,Cash,Online


In [13]:
conn.close()
print(f'Warehouse saved to {DB_FILE}')

Warehouse saved to ../warehouse/sales_warehouse.db
